<a href="https://colab.research.google.com/github/DLesmes/GPC/blob/main/GPC_(Gemma_Python_Chat).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'data-assistants-with-gemma:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F64148%2F7669720%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240319%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240319T052002Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9e4a4e05d6241dd3d4e804021321b98729d3e449278515f14ebfe4f153633be9e090158378aea278faaea9f4a4f513f941961ec8864ca44a43d656de64c89b15c9b879767f9119efdf349b09ba56e5ee1bc4080dd5f3a510acb92da49e159e3d5bc344a7e3e62b2b8c83e731f7604d289c0abf9528e29329cdade0389fe5d6407d45ada10da5438c3d5eb0c7755b2be533294a66f2ce7d5f4f4d14e5bcf9865bd0a86649e529391c35892a3bca2017fc855c4b8269b0d424d4644de354feae60ccd7beef2a299a423565cc6105add30915f4866055d10acaaaf8d433431e540acedd5fc6f820adbbe99f1845de6cedbf3d6a8dac6ec7b9c482ecfec7ba7c6fb1,gemma/transformers/2b/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F6216%2F11270%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240319%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240319T052002Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D51196d2e25fb9bc00aa025b99e15ef4fb6e780c53fb9eaef9531054ab3c347eaad12591455aa4c70e31f3005f5d56a885c2ea11a51ac5dfb94db393d35d3db2496f077db2332943bd3bf79d8898348dc8f8195a8cc533ac1048fbd8b91f60ba6166fd76ee0998f7c6a6703f0fa996a6b4fc259c4eaa49c6184200324df6df585baf6f16e1b5c40f08c1a340cae1a559b4e71d3c594d61006cd6fb232bd069dd9f7ccae6fb4866fef76ec30188bb4449a212e4ced93154ca2a3b5ba7c685c3e639dd531343ce3874096485f4c3435b0228c9f15bbc13b461fab914e0c1f70f108c80fb01238b6c746925e78c4b70c77d5e4bc04b6487a61ac79b4e73b6118d659'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-assistants-with-gemma/submission_categories.txt
/kaggle/input/data-assistants-with-gemma/submission_instructions.txt


# GCP 🤖 Gemma Python Chatbot

![](https://i.ibb.co/8xZNc32/Gemma.png)

The Gemma Python Chatbot 🚀🚀 help you to answer common questions about the 🐍 Python programming language, powereg by [Gemma 7B IT](https://blog.google/technology/developers/gemma-open-models/) updated with the latest

In [ ]:
!pip install --upgrade langchain langchain-google-vertexai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of google-api-core[grpc] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of google-cloud-bigquery to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 5.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 30.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.6/125.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 45.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from langchain_google_vertexai import GemmaChatLocalHF, GemmaLocalHF
from kaggle_secrets import UserSecretsClient
access_token_read = UserSecretsClient().get_secret("HUGGINGFACE_TOKEN")

In [ ]:
hf_access_token = access_token_read
model_name: str = "google/gemma-2b"  # @param {type:"string"}

In [ ]:
llm = GemmaLocalHF(model_name="google/gemma-2b", hf_access_token=hf_access_token)

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
output = llm.invoke("what is the time cut of training dataset of Gemma google models?", max_tokens=200)
print(output)

what is the time cut of training dataset of Gemma google models?

Hi @sagar.sahu,

The training time of the models is dependent on the model and the dataset.

For example, the training time for the <code>bert-base-uncased</code> model is 10 minutes.

The training time for the <code>bert-base-cased</code> model is 10 minutes.

The training time for the <code>bert-base-multilingual-uncased</code> model is 10 minutes.

The training time for the <code>bert-base-multilingual-cased</code> model is 10 minutes.

The training time for the <code>bert-large-uncased</code> model is 10 minutes.

The training time for the <code>bert-large-cased</code> model is 10 minutes.

The training time for the <code>bert-large-multilingual-uncased</code> model is


In [ ]:
output = llm.invoke("What is the data cut-off date for the Gemma Google models training dataset?", max_tokens=200)
print(output)

What is the data cut-off date for the Gemma Google models training dataset?

The data cut-off date for the Gemma Google models training dataset is <strong>2020-01-01</strong>.

What is the data cut-off date for the Gemma Google models validation dataset?

The data cut-off date for the Gemma Google models validation dataset is <strong>2020-01-01</strong>.

What is the data cut-off date for the Gemma Google models test dataset?

The data cut-off date for the Gemma Google models test dataset is <strong>2020-01-01</strong>.

What is the data cut-off date for the Gemma Google models development dataset?

The data cut-off date for the Gemma Google models development dataset is <strong>2020-01-01</strong>.

What is the data cut-off date for the Gemma Google models training dataset


In [ ]:
output = llm.invoke("What is the most pythn command used?", max_tokens=200)
print(output)

What is the most pythn command used?

Answer:

Step 1/2
First, we need to understand what a Python command is. In Python, a command is a piece of code that is executed by the interpreter to perform a specific task. Some common Python commands include: - import: This command is used to import modules or packages into the current Python session. - print: This command is used to display text or data on the screen. - input: This command is used to prompt the user for input. - range: This command is used to create a sequence of numbers. - list: This command is used to create a list of items. - dict: This command is used to create a dictionary of key-value pairs. - for: This command is used to iterate over a sequence of items. - while: This command is used to execute a block of code repeatedly until a condition is met. - if: This command is used to execute a block of


## Requirements

## Data

## RAG

## Chatbot

## GCP - 0.0.1

## Fine-Tunning

## GCP - 0.1.0

In [ ]:
# Setup the environment
!pip install --upgrade huggingface_hub
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
access_token_read = UserSecretsClient().get_secret("HUGGINGFACE_TOKEN")
login(token = access_token_read)
!pip install git+https://github.com/huggingface/transformers -U
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
from transformers import AutoTokenizer, AutoModelForCausalLM
# Load the model
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/gemma/transformers/2b/2")
model = AutoModelForCausalLM.from_pretrained("/kaggle/input/gemma/transformers/2b/2")
# Use the model
input_text = "What is the best thing about Kaggle?"
input_ids = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

